In [1]:
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from tqdm import tqdm

INPUT_DIR = '/home/quang/working/fundus_segmentation/data/segmentation_doctor/data_full/'
INPUT_DIR_MASK = '/home/quang/working/fundus_segmentation/data/segmentation_doctor/data_full/'

In [2]:
# Split data into train and test
list_names = glob.glob(INPUT_DIR + '*.jpg')
list_name_f = [x.split('/')[-1].split('.')[0] for x in list_names]
print (len(list_names))

list_name_f_train, list_name_f_test = train_test_split(list_name_f, test_size=0.15, random_state=42)
print (len(list_name_f_train), len(list_name_f_test))

775


### Crop images and mask for training dataset

In [ ]:
IMG_SIZE = 1200
PATCH_SIZE = 128

INPUT_DIR_PATCH_MASK_PRED = '/home/quang/working/fundus_segmentation/data/segmentation_doctor/data_full_mask_pred_DeepLabV3_ver2/'
OUTPUT_DIR = '/home/quang/working/fundus_segmentation/data/segmentation_doctor/patch_data_train_DeepLabV3_ver2/'
OUTPUT_MAKS_DIR = '/home/quang/working/fundus_segmentation/data/segmentation_doctor/patch_data_mask_pred_train_DeepLabV3_ver2/'

for file_name_temp in tqdm(list_name_f_train):
    img = cv2.imread(INPUT_DIR + file_name_temp + '.jpg')
    mask = cv2.imread(INPUT_DIR_MASK + file_name_temp + '.labels.tif', 0)
    
    mask_pred = cv2.imread(INPUT_DIR_PATCH_MASK_PRED + file_name_temp + '.jpg', 0)

    img_cropped = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_CUBIC)

    mask = np.where(mask > 0, 255, 0).astype(np.uint8)
    mask_cropped = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_CUBIC)
    mask_cropped = np.where(mask_cropped > 128, 255, 0).astype(np.uint8)

    mask_pred_cropped = cv2.resize(mask_pred, (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_CUBIC)
    
    img_gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
    img_gray_mask = np.where(img_gray > 10, 1, 0)

    circle = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.uint8)
    cv2.circle(circle, (int(IMG_SIZE/2), int(IMG_SIZE/2)), int(IMG_SIZE/2), 1, thickness=-1)

    img_gray_mask = img_gray_mask*circle

    temp = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.uint8)

    for i in range(0, IMG_SIZE-PATCH_SIZE, 100):
        for j in range(0, IMG_SIZE-PATCH_SIZE, 100):
            if img_gray_mask[i:i+PATCH_SIZE, j:j+PATCH_SIZE].sum() > PATCH_SIZE*PATCH_SIZE/2 and mask_cropped[i:i+PATCH_SIZE, j:j+PATCH_SIZE].sum() > 0:
                img_patch = img_cropped[i:i+PATCH_SIZE, j:j+PATCH_SIZE,:]
                mask_patch = mask_cropped[i:i+PATCH_SIZE, j:j+PATCH_SIZE]
                mask_pred_cropped_patch = mask_pred_cropped[i:i+PATCH_SIZE, j:j+PATCH_SIZE]
                cv2.imwrite(OUTPUT_DIR + file_name_temp + '_' + str(i) + '_' + str(j) + '.jpg',img_patch)
                cv2.imwrite(OUTPUT_DIR + file_name_temp + '_' + str(i) + '_' + str(j) + '.labels.tif',mask_patch)
                cv2.imwrite(OUTPUT_MAKS_DIR + file_name_temp + '_' + str(i) + '_' + str(j) + '.jpg',mask_pred_cropped_patch)

### Crop images and mask for testing dataset

In [7]:
IMG_SIZE = 1200
PATCH_SIZE = 128

INPUT_DIR_PATCH_MASK_PRED = '/home/quang/working/fundus_segmentation/data/segmentation_doctor/data_full_mask_pred_DeepLabV3_ver2/'
OUTPUT_DIR = '/home/quang/working/fundus_segmentation/data/segmentation_doctor/patch_data_test_DeepLabV3_ver2/'
OUTPUT_MAKS_DIR = '/home/quang/working/fundus_segmentation/data/segmentation_doctor/patch_data_mask_pred_test_DeepLabV3_ver2/'

for file_name_temp in tqdm(list_name_f_test):
    img = cv2.imread(INPUT_DIR + file_name_temp + '.jpg')
    mask = cv2.imread(INPUT_DIR_MASK + file_name_temp + '.labels.tif', 0)
    
    mask_pred = cv2.imread(INPUT_DIR_PATCH_MASK_PRED + file_name_temp + '.jpg', 0)

    img_cropped = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_CUBIC)

    mask = np.where(mask > 0, 255, 0).astype(np.uint8)
    mask_cropped = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_CUBIC)
    mask_cropped = np.where(mask_cropped > 128, 255, 0).astype(np.uint8)

    mask_pred_cropped = cv2.resize(mask_pred, (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_CUBIC)
    
    img_gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
    img_gray_mask = np.where(img_gray > 10, 1, 0)

    circle = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.uint8)
    cv2.circle(circle, (int(IMG_SIZE/2), int(IMG_SIZE/2)), int(IMG_SIZE/2), 1, thickness=-1)

    img_gray_mask = img_gray_mask*circle

    temp = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.uint8)

    for i in range(0, IMG_SIZE-PATCH_SIZE, 100):
        for j in range(0, IMG_SIZE-PATCH_SIZE, 100):
            if img_gray_mask[i:i+PATCH_SIZE, j:j+PATCH_SIZE].sum() > PATCH_SIZE*PATCH_SIZE/2 and mask_cropped[i:i+PATCH_SIZE, j:j+PATCH_SIZE].sum() > 0:
                img_patch = img_cropped[i:i+PATCH_SIZE, j:j+PATCH_SIZE,:]
                mask_patch = mask_cropped[i:i+PATCH_SIZE, j:j+PATCH_SIZE]
                mask_pred_cropped_patch = mask_pred_cropped[i:i+PATCH_SIZE, j:j+PATCH_SIZE]
                cv2.imwrite(OUTPUT_DIR + file_name_temp + '_' + str(i) + '_' + str(j) + '.jpg',img_patch)
                cv2.imwrite(OUTPUT_DIR + file_name_temp + '_' + str(i) + '_' + str(j) + '.labels.tif',mask_patch)
                cv2.imwrite(OUTPUT_MAKS_DIR + file_name_temp + '_' + str(i) + '_' + str(j) + '.jpg',mask_pred_cropped_patch)

100%|██████████| 78/78 [00:10<00:00,  7.36it/s]
